In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install segmentation_models -q
%env SM_FRAMEWORK=tf.keras

In [ ]:
import tensorflow as tf
import segmentation_models as sm
import os
import numpy as np
from matplotlib import pyplot as plt
from zipfile import ZipFile
from PIL import Image
from sklearn.model_selection import train_test_split
import albumentations as A
import gc

# Import tiled images

In [ ]:
# import data tiles (iafoss 512x512)

image_zip = '../input/iafoss-512x512/train.zip'
mask_zip = '../input/iafoss-512x512/masks.zip'

image_store = '../kaggle/working/train/'
mask_store = '../kaggle/working/masks/'

with ZipFile(image_zip, 'r') as imgs:
    imgs.extractall(image_store)

with ZipFile(mask_zip, 'r') as masks:
    masks.extractall(mask_store)

In [ ]:
TRAIN_TILES = []
TRAIN_MASKS = []

for tile in os.listdir(image_store):
    TRAIN_TILES.append(np.asarray(Image.open(image_store + tile)))

for mask in os.listdir(mask_store):
    TRAIN_MASKS.append(np.asarray(Image.open(mask_store + mask)))

# Data augmentation

In [ ]:
# define image augmentation steps
aug = A.Compose([# A.Normalize(mean=(0.6627, 0.5120, 0.6972), std=(0.1804, 0.2560, 0.1648),
                 #             max_pixel_value=255.0, always_apply=False, p=1.0),
                 A.OneOf([A.RandomRotate90(),
                          A.HorizontalFlip(),
                          A.VerticalFlip()], p=0.8),
                 A.OneOf([A.RandomBrightnessContrast(),
                          A.HueSaturationValue()], p=0.3)])

In [ ]:
aug = [aug(image=img, mask=mask) for (img, mask) in list(zip(TRAIN_TILES, TRAIN_MASKS))]

In [ ]:
X = [img['image'] for img in aug]
Y = [mask['mask'] for mask in aug]

In [ ]:
X = np.asarray(X)
Y = np.asarray(Y)

In [ ]:
del TRAIN_TILES, TRAIN_MASKS, aug
gc.collect()

# Build Model

In [ ]:
BACKBONE = 'resnet34'
BATCH_SIZE = 16
EPOCHS = 30

In [ ]:
preprocess_input = sm.get_preprocessing(BACKBONE)

In [ ]:
X_TRAIN, X_VAL, Y_TRAIN, Y_VAL = train_test_split(X, Y, test_size=0.2, random_state=15)

In [ ]:
del X, Y
gc.collect()

In [ ]:
X_TRAIN = preprocess_input(X_TRAIN)
X_VAL = preprocess_input(X_VAL)

In [ ]:
def dice_coe(output, target, axis = None, smooth=1e-10):
    output = tf.dtypes.cast( tf.math.greater(output, 0.5), tf. float32 )
    target = tf.dtypes.cast( tf.math.greater(target, 0.5), tf. float32 )
    inse = tf.reduce_sum(output * target, axis=axis)
    l = tf.reduce_sum(output, axis=axis)
    r = tf.reduce_sum(target, axis=axis)

    dice = (2. * inse + smooth) / (l + r + smooth)
    dice = tf.reduce_mean(dice, name='dice_coe')
    return dice

In [ ]:
model = sm.Unet(BACKBONE, encoder_weights='imagenet')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coe])
model.summary()

In [ ]:
check = tf.keras.callbacks.ModelCheckpoint(filepath='/kaggle/working/alina_resnet34.h5', verbose=1,
                                           save_best_only=True)

early_stop = tf.keras.callbacks.EarlyStopping(patience=7, monitor='val_dice_coe', mode='max',
                                              restore_best_weights=True)

In [ ]:
history = model.fit(X_TRAIN, Y_TRAIN, batch_size=BATCH_SIZE, epochs=EPOCHS,
                    callbacks=[check, early_stop], validation_data=(X_VAL, Y_VAL))

In [ ]:
# plot training and validation loss

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss)+1)
plt.plot(epochs, loss, 'y', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
model.save('alina_resnet34.h5')